In [1]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.Model.list()

AuthenticationError: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.

In [49]:
# Programs to train
def get_prompt_1(answer=False):
    possible_line_to_add ="\n+     assert |FlattenReverse(s[..|s|-1])| <= (|s|-1) * j;" if answer else ""

    p1 = f"""
lemma LemmaFlattenLengthLeMul<T>(s: seq<seq<T>>, j: int)
  requires forall i {{:trigger s[i]}} | 0 <= i < |s| :: |s[i]| <= j
  ensures |FlattenReverse(s)| <= |s| * j
  {{
    if |s| == 0 {{
    }} else {{
      LemmaFlattenLengthLeMul(s[..|s|-1], j);{possible_line_to_add}
    }}
  }}
    """
    return p1

get_prompt_1()

'\nlemma LemmaFlattenLengthLeMul<T>(s: seq<seq<T>>, j: int)\n  requires forall i {:trigger s[i]} | 0 <= i < |s| :: |s[i]| <= j\n  ensures |FlattenReverse(s)| <= |s| * j\n  {\n    if |s| == 0 {\n    } else {\n      LemmaFlattenLengthLeMul(s[..|s|-1], j);\n    }\n  }\n    '

In [59]:
def get_prompt_2(answer=False):
    possible_line_to_add = "\n+     assert ToNatRight(xs[..i+1]) != ToNatRight(ys[..i+1]);" if answer else ""
    p2 = f"""
lemma LemmaSeqPrefixNeq(xs: seq<uint>, ys: seq<uint>, i: nat)
    requires 0 <= i <= |xs| == |ys|
    requires ToNatRight(xs[..i]) != ToNatRight(ys[..i])
    ensures ToNatRight(xs) != ToNatRight(ys)
    decreases |xs| - i
  {{
    if i == |xs| {{
      assert xs[..i] == xs;
      assert ys[..i] == ys;
    }} else {{
      if xs[i] == ys[i] {{
        reveal ToNatLeft();
        assert DropLast(xs[..i+1]) == xs[..i];
        assert DropLast(ys[..i+1]) == ys[..i];
        LemmaToNatLeftEqToNatRightAuto();
        assert ToNatRight(xs[..i+1]) == ToNatLeft(xs[..i+1]);
      }} else if xs[i] < ys[i] {{
        LemmaSeqMswInequality(xs[..i+1], ys[..i+1]);
      }} else {{
        LemmaSeqMswInequality(ys[..i+1], xs[..i+1]);
      }}{possible_line_to_add}
      LemmaSeqPrefixNeq(xs, ys, i + 1);
    }}
  }}
    """
    return p2

print(get_prompt_2(answer=False))


lemma LemmaSeqPrefixNeq(xs: seq<uint>, ys: seq<uint>, i: nat)
    requires 0 <= i <= |xs| == |ys|
    requires ToNatRight(xs[..i]) != ToNatRight(ys[..i])
    ensures ToNatRight(xs) != ToNatRight(ys)
    decreases |xs| - i
  {
    if i == |xs| {
      assert xs[..i] == xs;
      assert ys[..i] == ys;
    } else {
      if xs[i] == ys[i] {
        reveal ToNatLeft();
        assert DropLast(xs[..i+1]) == xs[..i];
        assert DropLast(ys[..i+1]) == ys[..i];
        LemmaToNatLeftEqToNatRightAuto();
        assert ToNatRight(xs[..i+1]) == ToNatLeft(xs[..i+1]);
      } else if xs[i] < ys[i] {
        LemmaSeqMswInequality(xs[..i+1], ys[..i+1]);
      } else {
        LemmaSeqMswInequality(ys[..i+1], xs[..i+1]);
      }
      LemmaSeqPrefixNeq(xs, ys, i + 1);
    }
  }
    


In [51]:
def get_prompt_3(answer=False):
    p3_line_to_add ="""
+ calc {
+     Serialize(Deserialize(b));
+   == // Definitions of Serialize, Deserialize
+     Seq.Map(c => c as byte, Seq.Map(b => b as Utf8EncodingForm.CodeUnit, b));
+   == // Compositionality of Map
+     Seq.Map(b => (b as Utf8EncodingForm.CodeUnit) as byte, b);
+   == // Simplify map
+     Seq.Map(b => b, b);
+   == // Identity function
+    b;
+   }""" if answer else ""
    p3 = f"""
  type byte = BoundedInts.uint8
  /**
    * Returns the byte serialization of the given code unit sequence.
    */
  function Serialize(s: Utf8EncodingForm.CodeUnitSeq): (b: seq<byte>)
  {{
    Seq.Map(c => c as byte, s)
  }}
  /**
    * Returns the code unit sequence that serializes to the given byte sequence.
    */
  function Deserialize(b: seq<byte>): (s: Utf8EncodingForm.CodeUnitSeq)
  {{
    Seq.Map(b => b as Utf8EncodingForm.CodeUnit, b)
  }}
  /**
    * Serializing a code unit sequence and then deserializing the result, yields the original code unit sequence.
    */
  lemma LemmaSerializeDeserialize(s: Utf8EncodingForm.CodeUnitSeq)
    ensures Deserialize(Serialize(s)) == s
  {{
  }}
  /**
    * Deserializing a byte sequence and then serializing the result, yields the original byte sequence.
    */
  lemma LemmaDeserializeSerialize(b: seq<byte>)
    ensures Serialize(Deserialize(b)) == b
  {{{p3_line_to_add}
  }}
    """
    return p3

print(get_prompt_3(answer=True))


  type byte = BoundedInts.uint8
  /**
    * Returns the byte serialization of the given code unit sequence.
    */
  function Serialize(s: Utf8EncodingForm.CodeUnitSeq): (b: seq<byte>)
  {
    Seq.Map(c => c as byte, s)
  }
  /**
    * Returns the code unit sequence that serializes to the given byte sequence.
    */
  function Deserialize(b: seq<byte>): (s: Utf8EncodingForm.CodeUnitSeq)
  {
    Seq.Map(b => b as Utf8EncodingForm.CodeUnit, b)
  }
  /**
    * Serializing a code unit sequence and then deserializing the result, yields the original code unit sequence.
    */
  lemma LemmaSerializeDeserialize(s: Utf8EncodingForm.CodeUnitSeq)
    ensures Deserialize(Serialize(s)) == s
  {
  }
  /**
    * Deserializing a byte sequence and then serializing the result, yields the original byte sequence.
    */
  lemma LemmaDeserializeSerialize(b: seq<byte>)
    ensures Serialize(Deserialize(b)) == b
  {
+ calc {
+     Serialize(Deserialize(b));
+   == // Definitions of Serialize, Deserialize
+ 

In [52]:
system_prompt = """
You are a Dafny formal method expert. 
To fix proofs that are brittle or unstable you can add assertions
inspired from the code or from the ensure annotations.
"""

In [53]:
user_content_1 = f"""
Can you make this proof more stable:

{get_prompt_1()}
"""

assistant_content_1 = f"""
Absolutely, you can add an assertion inspired by the postcondition such as:
{get_prompt_1(answer=True)}
"""

print(assistant_content_1)


Absolutely, you can add an assertion inspired by the postcondition such as:

lemma LemmaFlattenLengthLeMul<T>(s: seq<seq<T>>, j: int)
  requires forall i {:trigger s[i]} | 0 <= i < |s| :: |s[i]| <= j
  ensures |FlattenReverse(s)| <= |s| * j
  {
    if |s| == 0 {
    } else {
      LemmaFlattenLengthLeMul(s[..|s|-1], j);
+     assert |FlattenReverse(s[..|s|-1])| <= (|s|-1) * j;
    }
  }
    



In [54]:
user_content_3 = f"""
Can you make this proof more stable:

{get_prompt_3()}
"""

assistant_content_3 = f"""
Absolutely, you can add assertions in a calc statement made by decomposing the operation using the other lemmas:
{get_prompt_3(answer=True)}
"""

print(assistant_content_3)


Absolutely, you can add assertions in a calc statement made by decomposing the operation using the other lemmas:

  type byte = BoundedInts.uint8
  /**
    * Returns the byte serialization of the given code unit sequence.
    */
  function Serialize(s: Utf8EncodingForm.CodeUnitSeq): (b: seq<byte>)
  {
    Seq.Map(c => c as byte, s)
  }
  /**
    * Returns the code unit sequence that serializes to the given byte sequence.
    */
  function Deserialize(b: seq<byte>): (s: Utf8EncodingForm.CodeUnitSeq)
  {
    Seq.Map(b => b as Utf8EncodingForm.CodeUnit, b)
  }
  /**
    * Serializing a code unit sequence and then deserializing the result, yields the original code unit sequence.
    */
  lemma LemmaSerializeDeserialize(s: Utf8EncodingForm.CodeUnitSeq)
    ensures Deserialize(Serialize(s)) == s
  {
  }
  /**
    * Deserializing a byte sequence and then serializing the result, yields the original byte sequence.
    */
  lemma LemmaDeserializeSerialize(b: seq<byte>)
    ensures Serialize(Des

In [60]:
user_content_2 = f"""
Can you make this proof more stable:

{get_prompt_2(answer=False)}
"""

print(user_content_2)


Can you make this proof more stable:


lemma LemmaSeqPrefixNeq(xs: seq<uint>, ys: seq<uint>, i: nat)
    requires 0 <= i <= |xs| == |ys|
    requires ToNatRight(xs[..i]) != ToNatRight(ys[..i])
    ensures ToNatRight(xs) != ToNatRight(ys)
    decreases |xs| - i
  {
    if i == |xs| {
      assert xs[..i] == xs;
      assert ys[..i] == ys;
    } else {
      if xs[i] == ys[i] {
        reveal ToNatLeft();
        assert DropLast(xs[..i+1]) == xs[..i];
        assert DropLast(ys[..i+1]) == ys[..i];
        LemmaToNatLeftEqToNatRightAuto();
        assert ToNatRight(xs[..i+1]) == ToNatLeft(xs[..i+1]);
      } else if xs[i] < ys[i] {
        LemmaSeqMswInequality(xs[..i+1], ys[..i+1]);
      } else {
        LemmaSeqMswInequality(ys[..i+1], xs[..i+1]);
      }
      LemmaSeqPrefixNeq(xs, ys, i + 1);
    }
  }
    



In [62]:
response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=[{
        "role": "system",
        "content": system_prompt
      },
      {
        "role": "user",
        "content": user_content_1
      },
      {
        "role": "assistant",
        "content": assistant_content_1
      },
      {
        "role": "user",
        "content": user_content_3
      },
      {
        "role": "assistant",
        "content": assistant_content_3
      },
      {
        "role": "user",
        "content": user_content_2
      }
    ],
  temperature=0,
  max_tokens=1024
)



In [73]:
answer = response["choices"][0]["message"]["content"]
print(answer)

Yes, you can add assertions to make the proof more stable. Here is the updated proof:

lemma LemmaSeqPrefixNeq(xs: seq<uint>, ys: seq<uint>, i: nat)
    requires 0 <= i <= |xs| == |ys|
    requires ToNatRight(xs[..i]) != ToNatRight(ys[..i])
    ensures ToNatRight(xs) != ToNatRight(ys)
    decreases |xs| - i
  {
    if i == |xs| {
      assert xs[..i] == xs;
      assert ys[..i] == ys;
    } else {
      if xs[i] == ys[i] {
        reveal ToNatLeft();
        assert DropLast(xs[..i+1]) == xs[..i];
        assert DropLast(ys[..i+1]) == ys[..i];
        LemmaToNatLeftEqToNatRightAuto();
        assert ToNatRight(xs[..i+1]) == ToNatLeft(xs[..i+1]);
+       assert ToNatRight(xs[..i+1]) != ToNatRight(ys[..i+1]); // Added assertion
      } else if xs[i] < ys[i] {
        LemmaSeqMswInequality(xs[..i+1], ys[..i+1]);
+       assert ToNatRight(xs[..i+1]) != ToNatRight(ys[..i+1]); // Added assertion
      } else {
        LemmaSeqMswInequality(ys[..i+1], xs[..i+1]);
+       assert ToNatRight(xs[.

In [75]:
def fix_proof(proof):
    user_question = f"""
Can you make this proof more stable:

{proof}
    """
    response = openai.ChatCompletion.create(
      model="gpt-4",
      messages=[{
        "role": "system",
        "content": system_prompt
      },
      {
        "role": "user",
        "content": user_content_1
      },
      {
        "role": "assistant",
        "content": assistant_content_1
      },
      {
        "role": "user",
        "content": user_content_3
      },
      {
        "role": "assistant",
        "content": assistant_content_3
      },
      {
        "role": "user",
        "content": user_question
      }
    ],
  temperature=0,
  max_tokens=1024
    )
    return response["choices"][0]["message"]["content"]

In [82]:
DIRECTORY = "./unstable_proofs"

def process_directory(directory_path):
    result_dict = {}

    # Iterate through each file in the directory
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)

        # Skip directories
        if os.path.isdir(file_path):
            continue

        with open(file_path, 'r') as file:
            # Read all lines
            lines = file.readlines()

            # Get the last line as the link
            link = lines[-1].strip()

            # Read all lines except the last one
            content_lines = lines[:-1]

            # Join the content lines into a string
            content = ''.join(content_lines)


            # Create a dictionary entry
            result_dict[filename] = {'content': content, 'link': link}

    return result_dict

proofs_to_fix = process_directory(DIRECTORY)

In [86]:
for proof in proofs_to_fix.values():
    proof_content = proof["content"]
    fix = fix_proof(proof_content)
    print(fix)
    print(f"PROOF LINK: {proof['link']}")
    print("=============================")

Yes, you can add assertions to make the proof more stable. Here is the updated proof:

```dafny
lemma {:vcs_split_on_every_assert} ProbUnifCorrectnessEvenCaseIff(n: nat, s: RNG, m: nat)
  requires m % 2 == 0
  requires n > 0
  ensures
    var a := ProbUnif(n / 2)(s).0;
    var b := Deconstruct(ProbUnif(n / 2)(s).1).0;
    ProbUnif(n)(s).0 == m <==> (!b && 2*a == m)
{
  var a := ProbUnif(n / 2)(s).0;
  var b := Deconstruct(ProbUnif(n / 2)(s).1).0;
  if ProbUnif(n)(s).0 == m {
    if (b && 2*a + 1 == m) {
      assert A: (2*a + 1) / 2 == 1 by {
        calc {
          (2*a + 1) / 2;
        == { LemmaAboutNatDivision(2*a + 1, 2); }
          ((2*a + 1) as real / 2 as real).Floor;
          (1 as real).Floor;
        ==
          1;
        }
      }
      assert m % 2 == 1 by { assert m == 2*a + 1; reveal A; assert m / 2 == 1 by { DivisionSubstituteAlternative(2, m, 2*a + 1); } }
      assert m % 2 == 0;
      assert false;
    }
    assert !(b && 2*a + 1 == m) ==> (!b && 2*a == m) by {

In [87]:
DIRECTORY_ROUND2= "./unstable_proofs/round2"
proofs_to_fix_2 = process_directory(DIRECTORY_ROUND2)

In [89]:
for proof in proofs_to_fix_2.values():
    proof_content = proof["content"]
    fix = fix_proof(proof_content)
    print(fix)
    print(f"PROOF LINK: {proof['link']}")
    print("=============================")

Sure, you can add an assertion to the lemma to make the proof more stable. The assertion can be derived from the definition of the unital property:

```dafny
lemma UnitIsUnique<T(!new)>(bop: (T, T) -> T, unit1: T, unit2: T)
  requires IsUnital(bop, unit1)
  requires IsUnital(bop, unit2)
  ensures unit1 == unit2
{
  assert bop(unit1, unit2) == unit2; // From IsRightUnital(bop, unit1)
  assert bop(unit1, unit2) == unit1; // From IsLeftUnital(bop, unit2)
}
```

The two assertions together imply that `unit1 == unit2`, which is the postcondition of the lemma.
PROOF LINK: https://dafny.org/latest/VerificationOptimization/VerificationOptimization
Yes, you can add assertions to check the intermediate results and the types of the expressions. Here is the modified proof:

```dafny
lemma InSingleFalseTypes(r: Request, s: EntityStore, e1: Expr, e2: Expr, t1: Type, t2: Type)
  requires subty(t1,Type.Entity(AnyEntity))
  requires subty(t2,Type.Entity(AnyEntity))
  requires IsSafe(r,s,e1,t1)
  requir